In [50]:
# doing an xg model based on the features extracted using MLlib
# shot_statsbomb_xg is the target variable, the rest are the features
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, DecisionTreeRegressor, GeneralizedLinearRegression, IsotonicRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
import pp_events as pp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.appName('Training').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 21:06:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# xgboost model
from xgboost import XGBRegressor
# XGBoost, LightGBM, CatBoost
#from pyspark.ml.classification import MultilayerPerceptronClassifier
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [24]:
events = spark.read.csv('Data/events.csv',header=True,inferSchema=True,sep=';')

In [32]:
columns_t = ['id','match_id','player_id','player','position',"shot_location_x", "shot_location_y",
             'distance_to_goal','shot_angle', 'shot_body_part','preferred_foot_shot', 
             'under_pressure','shot_statsbomb_xg','shot_outcome','goal']

In [26]:
#events_shot = pp.shot_data(events)
events_shot = events.filter((events.type=='Shot') & (events.period < 5))
events_shot = pp.split_location(events_shot)
events_shot = pp.distance_to_goal(events_shot)
events_shot = pp.get_shot_angle(events_shot)
events_shot = pp.shot_preferred_foot(events_shot,events)
events_shot = pp.goal(events_shot)

In [33]:
events_shot = events_shot.select(columns_t)

In [34]:
cols = ["under_pressure","preferred_foot_shot",'goal']
ev = pp.bool_to_int(events_shot,cols)

In [ ]:
ev = ev.na.fill(0)

In [51]:
features = ["shot_location_x", "shot_location_y", "distance_to_goal", "shot_angle", 'under_pressure', 'preferred_foot_shot']
assembler = VectorAssembler(inputCols=features, outputCol="features")
data = assembler.transform(ev).select("features", "goal", "shot_statsbomb_xg")

# Split into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [53]:
# Initialize models with label set to 'goal'
rf = RandomForestRegressor(labelCol="goal", featuresCol="features")
gbt = GBTRegressor(labelCol="goal", featuresCol="features")
dt = DecisionTreeRegressor(labelCol="goal", featuresCol="features")
glr = GeneralizedLinearRegression(labelCol="goal", featuresCol="features")
ir = IsotonicRegression(labelCol="goal", featuresCol="features")

# Create a list of models and their names
models = [rf, gbt, dt, glr, ir]
model_names = ["Random Forest", "Gradient Boosted Trees", "Decision Tree", "Generalized Linear Regression",
               "Isotonic Regression"]

# Create pipelines for each model
model_pipelines = [Pipeline(stages=[model]) for model in models]

In [54]:
# Train each model
fitted_models = [pipeline.fit(train_data) for pipeline in model_pipelines]

24/12/21 21:34:45 WARN Instrumentation: [7634b5e6] regParam is zero, which might cause numerical instability and overfitting.


In [55]:
# Evaluate on `shot_statsbomb_xg`
evaluator = RegressionEvaluator(labelCol="shot_statsbomb_xg", predictionCol="prediction", metricName="rmse")
rmse = []

for i, fitted_model in enumerate(fitted_models):
    # Generate predictions
    predictions = fitted_model.transform(test_data)
    
    # Map predicted probabilities to `shot_statsbomb_xg` range if applicable
    predictions = predictions.withColumn("prediction", col("prediction") * col("shot_statsbomb_xg"))  # Adjust based on logic
    
    # Evaluate RMSE
    model_rmse = evaluator.evaluate(predictions)
    rmse.append(model_rmse)
    print(model_names[i] + " RMSE: " + str(model_rmse))

Random Forest RMSE: 0.12211494856400057


Gradient Boosted Trees RMSE: 0.11530225458657559


Decision Tree RMSE: 0.1180496047719669


Generalized Linear Regression RMSE: 0.13063893956537284


Isotonic Regression RMSE: 0.14656223085733164


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GPU Check") \
    .config("spark.executor.resource.gpu.amount", "1")\
    .config("spark.task.resource.gpu.amount", "1") \
    .config("spark.executorEnv.LD_LIBRARY_PATH", "/usr/local/cuda/lib64") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 21:44:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/21 21:44:32 WARN ResourceUtils: The configuration of cores (exec = 12 task = 1, runnable tasks = 12) will result in wasted resources due to resource gpu limiting the number of runnable tasks per executor to: 1. Please adjust your configuration.


In [ ]:
# Time spent to run : 1m 54s for ["shot_location_x", "shot_location_y", "distance_to_goal", "shot_angle"] 0.09
# Time spent to run : 1m 50s for ["distance_to_goal", "shot_angle"]                                       0.10

# Consider using Classification instead of Regression